In [75]:
from fastai import *
from fastai.vision import *

In [76]:
# to get all files from a directory
import os

# to easier work with paths
from pathlib import Path

# to read and manipulate .csv-files
import pandas as pd

In [77]:
INPUT = Path("./input/digit-recognizer")
os.listdir(INPUT)

['test.csv', 'train.csv']

In [78]:
train_df = pd.read_csv(INPUT/"train.csv")
train_df.head(3)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
test_df = pd.read_csv(INPUT/"test.csv")
test_df.head(3)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
TRAIN = Path("./train")
TEST = Path("./test")

In [81]:
# # Create training directory
# for index in range(10):
#     try:
#         os.makedirs(TRAIN/str(index))
#     except:
#         pass

In [82]:
# #Create test directory
# try:
#     os.makedirs(TEST)
# except:
#     pass

In [83]:
# # import numpy to reshape array from flat (1x784) to square (28x28)
# import numpy as np

# # import PIL to display images and to create images from arrays
# from PIL import Image

# def saveDigit(digit, filepath):
#     digit = digit.reshape(28,28)
#     digit = digit.astype(np.uint8)

#     img = Image.fromarray(digit)
#     img.save(filepath)

In [84]:
# # save training images
# for index, row in train_df.iterrows():
    
#     label,digit = row[0], row[1:]
    
#     folder = TRAIN/str(label)
#     filename = f"{index}.jpg"
#     filepath = folder/filename
    
#     digit = digit.values
    
#     saveDigit(digit, filepath)

In [85]:
# # save testing images
# for index, digit in test_df.iterrows():

#     folder = TEST
#     filename = f"{index}.jpg"
#     filepath = folder/filename
    
#     digit = digit.values
    
#     saveDigit(digit, filepath)

In [86]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
import torch

class CustomImageDataset(Dataset):
    def __init__(self, input_dir: Path, label_dir: Path, transform=None):
        self.input_dir = input_dir
        self.label_dir = label_dir
        self.transform = transform

    def __len__(self):
        length = 0
        for digit_dir in self.input_dir.iterdir():
            length += len(list(digit_dir.glob('*')))
        return length
        # return len([os.listdir(digit_dir) for digit_dir in self.input_dir.iterdir()])

    def _dir_checker(self, idx: int) -> tuple:
        length = 0
        for digit_dir in self.input_dir.iterdir():
            act_length = length + len(list(digit_dir.glob('*')))
            if act_length >= idx + 1:
                return (digit_dir, idx - length)
            length = act_length
        raise IndexError(f"not supossed to be here: {idx, length}")

    def __getitem__(self, idx):
        work_dir_path, new_id = self._dir_checker(idx)
        input_path = list(work_dir_path.glob('*'))[new_id]
        image = read_image(str(input_path))

        label_name = work_dir_path.with_suffix('.png').name
        label_path = self.label_dir / label_name
        label = read_image(str(label_path))
        if self.transform:
            image = self.transform(image)
        return image, label

In [87]:
INPUT_PATH = Path('train')
LABEL_PATH = Path('label')

dset = CustomImageDataset(INPUT_PATH, LABEL_PATH)

In [88]:
len(dset)

42000

In [89]:
def split_calculator(dataset, valid_per):
    number_of_file = len(dataset)
    split_length = [valid_per*number_of_file, (1-valid_per)*number_of_file]
    return [round(split_length[0]), round(split_length[1])]

In [90]:
VALID_PER = 0.8
SEED = 42
BATCH_SIZE = 32
epochs = 10
IMG_SIZE = 120

In [91]:
split_length = split_calculator(dset, VALID_PER)
split_length = split_calculator(dset, VALID_PER)
training_dset, validation_dset = torch.utils.data.random_split(
    dset, 
    split_length, 
    generator=torch.Generator().manual_seed(SEED)
    )

In [92]:
training_dls = torch.utils.data.DataLoader(training_dset, batch_size=BATCH_SIZE, shuffle=True)
validation_dls = torch.utils.data.DataLoader(validation_dset, batch_size=BATCH_SIZE)            

In [93]:
# dls = fastai.data.all.DataLoaders(training_dls, validation_dls)

NameError: name 'fastai' is not defined

In [94]:
class LSNet(torch.nn.Module):
    def __init__(self):
        # call the parent constructor
        super(LSNet, self).__init__()
        self.conv_part = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, kernel_size=5, stride=2, padding=2),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(inplace=True),
        )
        self.deconv_part = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Upsample(scale_factor=2, mode='bilinear'),
            torch.nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Upsample(scale_factor=2, mode='bilinear'),
            torch.nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Upsample(size=IMG_SIZE, mode='bilinear'),
        )
    def forward(self, x):
        x = self.conv_part(x)
        x = self.deconv_part(x)
        return x

In [95]:
model = LSNet()

In [96]:
input_test = torch.rand(20, 1, IMG_SIZE, IMG_SIZE)
print(input_test.shape)
model(input_test).shape

torch.Size([20, 1, 120, 120])


torch.Size([20, 1, 120, 120])

In [97]:
from torchsummary import summary
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 14, 14]             416
       BatchNorm2d-2           [-1, 16, 14, 14]              32
              ReLU-3           [-1, 16, 14, 14]               0
            Conv2d-4             [-1, 32, 7, 7]           4,640
       BatchNorm2d-5             [-1, 32, 7, 7]              64
              ReLU-6             [-1, 32, 7, 7]               0
            Conv2d-7             [-1, 64, 4, 4]          18,496
       BatchNorm2d-8             [-1, 64, 4, 4]             128
              ReLU-9             [-1, 64, 4, 4]               0
  ConvTranspose2d-10             [-1, 32, 7, 7]          18,464
             ReLU-11             [-1, 32, 7, 7]               0
         Upsample-12           [-1, 32, 14, 14]               0
  ConvTranspose2d-13           [-1, 16, 27, 27]           4,624
             ReLU-14           [-1, 16,

In [98]:
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [99]:
def train(dataloader, model, loss_fn, optimizer):
    r_loss = 0
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 8 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
        r_loss = loss
    return r_loss

In [100]:
def test(dataloader, model, loss_fn):
    losses = []
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            # X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")
    return test_loss
    

In [101]:
import time
def time_printer(start_time, loop_start_time, count, epochs):
        run_time = time.time() - start_time
        run_time_str = time.strftime('%H:%M:%S', time.gmtime(int(run_time)))
        loop_time = time.time() - loop_start_time
        full_time_est = (run_time / (count+1))*epochs
        full_time_str = time.strftime('%H:%M:%S', time.gmtime(int(full_time_est)))
        print("-"*30)
        print(f"Processed: {count+1} / {epochs}\nRuntime:{run_time_str}\nLooptime:{loop_time:.2f}\nEstimated runtime:{full_time_str} ")
        print("-"*30)

In [102]:
train_losses, test_losses = [], []
start_time = time.time()
for t in range(epochs):
    loop_start_time = time.time()
    print(f"Epoch {t+1}\n-------------------------------")
    tr_loss = train(training_dls, model, loss_fn, optimizer)
    te_loss = test(validation_dls, model, loss_fn)
    train_losses.append(tr_loss)
    test_losses.append(te_loss)
    time_printer(start_time, loop_start_time, t, epochs)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: Input type (unsigned char) and bias type (float) should be the same